In [ ]:
# ==============================================================
# 0. Imports & environment
# ==============================================================
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.linalg import condition, pinv
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)


In [ ]:
# ==============================================================
# 1. Load a public proxy dataset (UCI Thyroid Cancer Recurrence)
# ==============================================================
# Download from: https://archive.ics.uci.edu/ml/machine-learning-databases/00615/
# File name: thyroid_recurrence.csv (383 rows, 12 features + target)
df = pd.read_csv('thyroid_recurrence.csv')
print(f"Raw shape: {df.shape}")

# Target: Recurred (Yes/No) → binary 1/0
df['Recurred'] = (df['Recurred'] == 'Yes').astype(int)
y = df['Recurred'].values
X_raw = df.drop('Recurred', axis=1)

# One-hot encode categorical columns (paper assumes numeric input)
X = pd.get_dummies(X_raw, drop_first=True)
print(f"After one-hot: {X.shape}")

In [ ]:
# ==============================================================
# 2. Missing-data handling + conditioning check
# ==============================================================
# Paper: whole-batch conditioning is better than mini-batch.
imputer = SimpleImputer(strategy='mean')
X_filled = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

def batch_conditioning(X_df, batch_size=32):
    conds = []
    for i in range(0, len(X_df), batch_size):
        batch = X_df.iloc[i:i+batch_size].values
        if batch.shape[0] > 1 and np.linalg.matrix_rank(batch) > 0:
            conds.append(condition(batch))
    return np.mean(conds) if conds else np.inf

mini_cond = batch_conditioning(X_filled, batch_size=32)
whole_cond = condition(X_filled.values)
print(f"Mini-batch cond ≈ {mini_cond:,.1f} | Whole-batch cond ≈ {whole_cond:,.1f}")

In [ ]:
# ==============================================================
# 3. Dimensionality reduction
# ==============================================================
# 3a. Inner-similarity: drop highly correlated features (|ρ|>0.8)
corr = X_filled.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [c for c in upper.columns if any(upper[c] > 0.8)]
X_inner = X_filled.drop(to_drop, axis=1)
print(f"Inner reduction → {X_inner.shape[1]} features (dropped {len(to_drop)})")

# 3b. Target-similarity: mutual information top-k
mi = mutual_info_classif(X_inner, y, random_state=42)
k = min(10, X_inner.shape[1])                 # keep at most 10
top_idx = mi.argsort()[-k:][::-1]
X_target = X_inner.iloc[:, top_idx]
print(f"Target reduction → {X_target.shape[1]} features")

X_reduced = X_target

In [ ]:
# ==============================================================
# 4. Size reduction – hierarchical clustering
# ==============================================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_reduced)

Z = linkage(X_scaled, method='weighted', metric='euclidean')
clusters = fcluster(Z, t=1.5, criterion='distance')   # tune t for ~30-50% reduction

unique = np.unique(clusters)
keep_idx = []
for cid in unique:
    mask = clusters == cid
    centroid = X_scaled[mask].mean(axis=0)
    dists = np.linalg.norm(X_scaled[mask] - centroid, axis=1)
    keep_idx.append(np.where(mask)[0][dists.argmin()])

X_final = X_reduced.iloc[keep_idx].reset_index(drop=True)
y_final = y[keep_idx]
print(f"Size reduction → {X_final.shape[0]} samples (kept {len(keep_idx)})")

In [ ]:
# ==============================================================
# 5. Train / test split
# ==============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X_final.values, y_final, test_size=0.2, stratify=y_final, random_state=42)

# Torch tensors (float32)
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_t  = torch.tensor(X_test,  dtype=torch.float32)
y_test_t  = torch.tensor(y_test,  dtype=torch.float32).view(-1, 1)

In [ ]:
# ==============================================================
# 6. ALGORITHM IMPLEMENTATIONS + MATHEMATICAL EXPLANATION
# ==============================================================

# ------------------------------------------------------------------
# 6.1 Batch Least Squares (BLS) – closed-form linear regression
# ------------------------------------------------------------------
"""
BLS solves   y = X w + b   in the least-squares sense, then applies
sigmoid for binary classification.

Math:
    w* = (XᵀX)⁻¹ Xᵀy          (Moore-Penrose if singular)
    b* = mean(y - X w*)
    ŷ  = σ(X w* + b*)         σ(z) = 1/(1+e⁻ᶻ)
"""
W_bls = pinv(X_train @ X_train.T) @ X_train @ y_train   # shape (n,1) → (d,1)
b_bls = (y_train - X_train @ W_bls).mean()
logits = X_test @ W_bls + b_bls
y_pred_bls = (torch.sigmoid(torch.tensor(logits)) > 0.5).float()
acc_bls = accuracy_score(y_test, y_pred_bls)
print(f"[BLS] Test accuracy: {acc_bls:.1%}")

# ------------------------------------------------------------------
# 6.2 Iterative Neural Network (INN) – 1-layer perceptron with GD
# ------------------------------------------------------------------
"""
A single linear layer + sigmoid, trained with stochastic gradient descent.
Equivalent to logistic regression but implemented as a tiny NN.

Loss:  L = BCE(σ(X w + b), y)
Update: w ← w - η ∇L   (SGD)
"""
class INN(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.linear = nn.Linear(dim, 1)
        nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

inn = INN(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.SGD(inn.parameters(), lr=0.05, momentum=0.9)

for epoch in range(300):
    optimizer.zero_grad()
    out = inn(X_train_t)
    loss = criterion(out, y_train_t)
    loss.backward()
    optimizer.step()

with torch.no_grad():
    y_pred_inn = (inn(X_test_t) > 0.5).float()
acc_inn = accuracy_score(y_test_t, y_pred_inn)
print(f"[INN] Test accuracy: {acc_inn:.1%}")

# ------------------------------------------------------------------
# 6.3 Least Squares with Linear Constraints (LSLC)
# ------------------------------------------------------------------
"""
Same linear model as BLS but adds *non-negativity* on weights
(and optionally L2 penalty). Paper uses a constrained optimizer.

We implement a simple projected gradient descent:
   w ← max(w - η ∇L, 0)
"""
class LSLC(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.w = nn.Parameter(torch.randn(dim, 1))
        self.b = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return torch.sigmoid(x @ self.w + self.b)

lslc = LSLC(X_train.shape[1])
opt = optim.SGD(lslc.parameters(), lr=0.02)

for epoch in range(400):
    opt.zero_grad()
    out = lslc(X_train_t)
    loss = nn.BCELoss()(out, y_train_t)
    loss.backward()
    opt.step()
    # Projection: enforce w ≥ 0
    with torch.no_grad():
        lslc.w.clamp_(min=0.0)

with torch.no_grad():
    y_pred_lslc = (lslc(X_test_t) > 0.5).float()
acc_lslc = accuracy_score(y_test_t, y_pred_lslc)
print(f"[LSLC] Test accuracy: {acc_lslc:.1%}")

# ------------------------------------------------------------------
# 6.4 Imperialistic Competitive Algorithm (ICA) – population optimizer
# ------------------------------------------------------------------
"""
Meta-heuristic inspired by socio-political competition.
- Population = (countries) each holding a weight vector w + bias b
- Imperialists = best countries
- Colonies move toward their imperialist (assimilation)
- Revolution & competition eliminate weak empires

Fitness = 1 / (1 + MSE)   (higher → better)
"""
def sigmoid(z): return 1/(1+np.exp(-z))

def mse_error(X, y, params):
    w, b = params[:-1], params[-1]
    pred = sigmoid(X @ w + b)
    return np.mean((pred - y)**2)

def ica_optimize(X, y, pop_size=60, max_iter=30, dim=X.shape[1]):
    # ---- initialization ----
    pop = np.random.uniform(-2, 2, (pop_size, dim+1))   # w + b
    costs = np.array([mse_error(X, y, p) for p in pop])
    
    for it in range(max_iter):
        # sort & assign imperialists (top 10%)
        idx = np.argsort(costs)
        pop, costs = pop[idx], costs[idx]
        n_imp = max(1, pop_size//10)
        imperialists = pop[:n_imp]
        imp_costs    = costs[:n_imp]

        # ---- assimilation ----
        new_pop = pop.copy()
        for i in range(n_imp, pop_size):
            imp = imperialists[np.random.randint(n_imp)]
            beta = np.random.uniform(0.1, 0.9)
            new_pop[i] = pop[i] + beta * (imp - pop[i])
            # small random revolution
            if np.random.rand() < 0.05:
                new_pop[i] += np.random.normal(0, 0.2, new_pop[i].shape)

        # ---- evaluate new population ----
        new_costs = np.array([mse_error(X, y, p) for p in new_pop])
        
        # ---- competition (replace worst imperialist if a colony is better) ----
        for i in range(n_imp):
            best_colony_idx = n_imp + np.argmin(new_costs[n_imp:])
            if new_costs[best_colony_idx] < imp_costs[i]:
                # swap
                imperialists[i], new_pop[best_colony_idx] = new_pop[best_colony_idx], imperialists[i]
                imp_costs[i], new_costs[best_colony_idx] = new_costs[best_colony_idx], imp_costs[i]

        pop, costs = new_pop, new_costs

    best = pop[np.argmin(costs)]
    w_best, b_best = best[:-1], best[-1]
    test_pred = sigmoid(X_test @ w_best + b_best)
    y_pred_ica = (test_pred > 0.5).astype(float)
    acc = accuracy_score(y_test, y_pred_ica)
    print(f"[ICA] Test accuracy: {acc:.1%}")
    return best

ica_optimize(X_train, y_train)

In [ ]:
# ==============================================================
# 7. Summary of results
# ==============================================================
print("\n=== FINAL SUMMARY ===")
print(f"BLS   : {acc_bls:.1%}")
print(f"INN   : {acc_inn:.1%}")
print(f"LSLC  : {acc_lslc:.1%}")
# ICA printed inside function